In [ ]:
from Load_Predictions import *
from downstream_model import *
from Featurize import *
from sklearn import metrics
from sklearn.metrics import confusion_matrix, accuracy_score
import copy
import warnings
import random
import matplotlib.pyplot as plt
import itertools
import numpy as np
import pandas as pd
import copy
import math
import scipy
import time
import h2o
from h2o.estimators.random_forest import H2ORandomForestEstimator

warnings.filterwarnings("ignore")
random.seed(100)

In [ ]:
### Specify the input file name, feature type annotations, and the name of the duplicate and target column

InputFilePath = 'Etailing/etailing.csv'
LabelFile = 'labels/dataFeaturized-etailing.csv'

duplicateColumn = 'State'
TargetColumn = 'What is the maximum cart value you ever shopped?'

### when using similarity encoding, just set the flag to 1. 0 means one-hot encoding.
similarity_flag = 0 

In [ ]:
dataDownstream = pd.read_csv(InputFilePath)
dataDownstream = dataDownstream.sample(frac=1, random_state=100)

dataDownstream = dataDownstream[dataDownstream[TargetColumn].notna()]
dataDownstream = dataDownstream[dataDownstream[duplicateColumn].notna()]

dataDownstream = dataDownstream.fillna('0')
dataDownstream = dataDownstream.reset_index(drop=True)

y = dataDownstream[[TargetColumn]]
dataDownstream = dataDownstream.drop(TargetColumn, axis=1)

In [ ]:
dataFeaturized = FeaturizeFile(dataDownstream)

In [ ]:
attribute_names = dataDownstream.columns.values.tolist()

attribute_dic = {}
labelsfile = pd.read_csv()

for index,row in labelsfile.iterrows(): attribute_dic[row['Attribute_name']] = row['label']
attribute_dic[duplicateColumn] = 10
print(attribute_dic)

y_cur = []
for x in attribute_names: y_cur.append(attribute_dic[x])

# Results on Raw

In [ ]:
# Evaluate One-hot Encoding
lrm,avgsc_train_lst_LR,avgsc_lst_LR,avgsc_hld_lst_LR = LogRegClassifier(dataDownstream,y,y_cur,attribute_names,similarity_flag)
rfm,avgsc_train_lst_RF,avgsc_lst_RF,avgsc_hld_lst_RF = RandForestClassifier(dataDownstream,y,y_cur,attribute_names,similarity_flag)

In [ ]:
h2o.init()

In [ ]:
dataDownstream_h2o = copy.deepcopy(dataDownstream)
y_h2o = copy.deepcopy(y)
y_h2o.update('"' + y_h2o.astype(str) + '"')

In [ ]:
# Evaluate String Encoding
rfm_h2o, avgsc_train_lst_h2o,avgsc_lst_h2o,avgsc_hld_lst_h2o =  RandForestH2oClassifier(dataDownstream_h2o, y_h2o, y_cur,attribute_names,TargetColumn,similarity_flag)

# Results on Deduped

In [ ]:
df = pd.read_csv(InputDownPath)
df = df.sort_values(by=['times_entered'], ascending=False)
df = df.sort_values(by=['group'])
df = df.reset_index()
df

In [ ]:
prvgrpno = 0
dicdups = {}
curdup = ''
for index, row in df.iterrows():
    curgrpno = row['group']
    if curgrpno!= curgrpno:
        curdup = row[duplicateColumn]
        dicdups[curdup] = prvgrpno
        prvgrpno = prvgrpno + 1
    else:
        if prvgrpno != curgrpno: 
            curdup = row[duplicateColumn]
            dicdups[curdup] = curdup
        else: dicdups[row[duplicateColumn]] = curdup
        prvgrpno = curgrpno

print(len(dicdups))
print(len(set(dicdups.values())))
print(dicdups)

In [ ]:
dataDownstream_dedup = copy.deepcopy(dataDownstream)
dataDownstream_dedup[duplicateColumn] = dataDownstream_dedup[duplicateColumn].fillna('0')

CleanedduplicateColumn = 'Duplicate_CleanCol'
dataDownstream_both = copy.deepcopy(dataDownstream)
dataDownstream_both[CleanedduplicateColumn] = dataDownstream_both[duplicateColumn]
dataDownstream_both[CleanedduplicateColumn] = dataDownstream_both[CleanedduplicateColumn].fillna('0')

def func(x):
    if x == '0': return '0'
    try: return dicdups[x]
    except KeyError: return '0'
    
dataDownstream_dedup[duplicateColumn] = dataDownstream_dedup[duplicateColumn].apply(lambda x: func(x))
dataDownstream_both[CleanedduplicateColumn] = dataDownstream_both[CleanedduplicateColumn].apply(lambda x: func(x))

dataFeaturized_dedup = FeaturizeFile(dataDownstream_dedup)
dataFeaturized_both = FeaturizeFile(dataDownstream_both)

attribute_names = dataDownstream_dedup.columns.values.tolist()
attribute_names_both = dataDownstream_both.columns.values.tolist()

y_cur_both = copy.deepcopy(y_cur)
y_cur_both.append(1)

In [ ]:
lrm_dedup,avgsc_train_lst_LR_dedup,avgsc_lst_LR_dedup,avgsc_hld_lst_LR_dedup = LogRegClassifier(dataDownstream_dedup,y,y_cur,attribute_names,similarity_flag)
rfm_dedup,avgsc_train_lst_RF_dedup,avgsc_lst_RF_dedup,avgsc_hld_lst_RF_dedup = RandForestClassifier(dataDownstream_dedup,y,y_cur,attribute_names,similarity_flag)
bestPerformingModel, avgsc_train_lst,avgsc_lst,avgsc_hld_lst = RandForestH2oClassifier(dataDownstream_dedup, y_h2o, y_cur,attribute_names,TargetColumn,similarity_flag)
